<!-- Auto table of contents -->
<h1 class='tocIgnore'>Visionature : DS via points d'écoute printaniers</h1>
<p>N.B. A partir de la version 0.9.2 (22/01/2022), pyaudisam peut-être utilisé en ligne de commande pour rendre
   les mêmes services que ce Notebook à partir du chapitre <a href="#XII.-Chargement-des-donn%C3%A9es-pr%C3%AAtes-pour-l'analyse-DS">XII</a> (exemple: Cf. <a href="./donnees/acdc/acdc-2019-ds-params.py">acdc-2019-ds-params.py</a>).</p>

# Imports

In [ ]:
import sys
import os
import shutil
import re

import pathlib as pl
import datetime as dt

import pandas as pd
import pandas.api.types as pdt
import numpy as np

from IPython.display import HTML

In [ ]:
# Comment-out to use "official" site-packages pyaudisam version.
sys.path.insert(0, '../pyaudisam')  # Or not ... to use local  development tree

import pyaudisam as ads

print('pyaudisam', ads.__version__, 'from', pl.Path(ads.__path__[0]).resolve().as_posix())

ads.runtime

# Communs

In [ ]:
# Create temporary directory if not yet done.
tmpDir = pl.Path('tmp')
tmpDir.mkdir(exist_ok=True)

In [ ]:
# Logging configuration.
ads.log.configure(handlers=[sys.stdout, tmpDir / 'vndspt.log'], reset=True,
                  loggers=[dict(name='matplotlib', level=ads.WARNING),
                           dict(name='ads', level=ads.INFO2),
                           dict(name='ads.eng', level=ads.INFO),
                           dict(name='ads.exr', level=ads.INFO),
                           #dict(name='ads.dat', level=ads.DEBUG),
                           #dict(name='ads.rep', level=ads.DEBUG),
                           dict(name='ads.anr', level=ads.DEBUG1),
                           dict(name='ads.onr', level=ads.DEBUG),
                           dict(name='visionat', level=ads.INFO),
                           dict(name='geocarto', level=ads.INFO),
                           dict(name='vndspt', level=ads.DEBUG)])

logger = ads.logger('vndspt')

In [ ]:
def backup(fpn, to='.', tsFmt='.%y%m%d'):
    """Backup given file to target folder with custom-formatted timestamp in name"""
    fpn = pl.Path(fpn)
    tn = fpn.stem + pd.Timestamp.now().strftime(tsFmt) + fpn.suffix
    tp = pl.Path(to) if to != '.' else fpn.parent
    logger.info('Backing up to ' + (tp / tn).as_posix())
    shutil.copy(fpn, tp / tn)

# I. Paramètres de l'étude : import / filtrage des données, ...

* fichier source provenant d'un export VisioNature,
* source Faune France ou Faune Auvergne (avec éventuellement qq colonnes en plus préparées ailleurs)
* colonnes à conserver, ignorer, renommer,
* lignes à ignorer
* ...

In [ ]:
# Par défaut.
# a. VisionatureDataSet
feuille = 0
source = 'FA' # Faune-Auvergne
ignorerLignes = []

colLongTrace = None
avecTraces = False

renommerCols = dict()
calculerCols = dict()

colEspece = 'Espèce'
colPassage = 'Passage'
colDistance = 'Distance'

garderCols = ['ID liste', 'Liste complète ?', 'Commentaire de la liste',
              'Date', 'Ref', 'Horaire', 'Lieu-dit', 'Commune', colEspece,
              'Estimation', 'Nombre', 'Détails', 'Code atlas',
              'Lat (WGS84)', 'Lon (WGS84)', 'Remarque']
garderColsListes = []

dCategoriesImbriquees = dCategoriesImbriqueesTout = dict()

# b. Pour simple affichage
obsBrutesColsAff = ['Date', 'ID liste', 'Point', 'Ref', 'Horaire', colEspece, 'Nombre', 'Détails', 'Code atlas']
comptesColsAff = ['nMalAd', 'nAutAd', 'nJuv', 'nVol']

# c. FieldDataSet
inventaireCols = ['Observateur', 'ID liste', 'Date', colPassage, 'Point']

effectifCols = ['nMalAd', 'nAutAd']  # Colonnes d'effectifs à prendre en compte (on ignore les autres)

def categorieAdulte(sEffectifs):  # Calcul catégorie "Adulte" = _m_âle ou _a_utre.
    return 'm' if 'Mal' in sEffectifs[sEffectifs > 0].index[0] else 'a'

ajouterMonoCatCols = dict(Adulte=categorieAdulte)

obsIndivCols = ['Observateur', 'ID liste', 'Date', colPassage, 'Point',
                'Ref', 'Horaire', colEspece, 'Adulte', colDistance]

# d. Analyse DS
distanceUnit = 'Meter'
areaUnit = 'Sq. Kilometer'
surveyType = 'Point'
distanceType = 'Radial'

groupage = False
colEffort = 'Effort'
effortConst = 1 # Valeur d'effort constante = 1 par passage sur chaque point.
colsSpeSelEchant = ['Adulte']  # Colonnes de sélection des échantillons : en plus de Espèce et Passage. 

# e. Carto, rapports d'analyses ...
nomEtude = None
sousEtude = ''
fusionnerSousEtudes = []
titreEtude = None
descrEtude = '<pas de description>'
motsClesEtude = ''

# f. Divers

# Fin de Par défaut.

## ACDC 2019

### Commun à toutes les sous études ACDC

In [ ]:
dossier = pl.Path('donnees/acdc')

# a. VisionatureDataSet et stats échantillons

def categoriesDuree(sObs):
    return ['10mn'] if sObs['Minute'] >= 5 else ['5mn', '10mn']
dCategoriesImbriquees = {'Durée': categoriesDuree}
dCategoriesImbriqueesTout = {'Durée': '10mn'}

# d. Analyse DS
distanceUnit = 'Meter'
areaUnit = 'Sq. Kilometer'
surveyType = 'Point'
distanceType = 'Radial'

dZoneEtude = dict(Zone='ACDC', Surface=24) # km2
colsSpeSelEchant = ['Adulte', 'Durée']  # Colonnes de sélection des échantillons : en plus de Espèce et Passage. 

# e. Carto, rapports d'analyses ...
nomEtude = 'ACDC2019'

### a. Naturalist

(source : exports FA individuels rassemblés, nettoyés, filtrés et enrichis des colonnes 'Passage', 'Point', 'Minute'
 via [NB ACDC-donnees-Naturalist](./ACDC-donnees-Naturalist.ipynb))

In [ ]:
# a. VisionatureDataSet

# Exports FA individuels rassemblés, nettoyés, filtrés et enrichis des colonnes 'Passage', 'Point', 'Minute'
# via NB ACDC-donnees-naturalist.
ficSrcVN = [dossier / 'ACDC2019-Naturalist-ObsBrutesNettoyees.xlsx']

source = 'FA' # Faune-Auvergne
avecTraces = False

renommerCols = {'Nom latin': colEspece}  # Attention : 'Nom scientifique' sur FF

def idObservateur(sObs):
    return sObs['Prénom'] + ' ' + sObs['Nom']
def minuteDInventaire(sObs):
    dateHeure = dt.datetime.combine(sObs['Date'].date(), dt.time.fromisoformat(sObs['Horaire']))
    return (dateHeure - sObs['Heure début']).total_seconds() / 60
calculerCols = dict(Observateur=idObservateur, Minute=minuteDInventaire)

garderCols += ['Minute']
garderColsListes = ['Observateur', colPassage, 'Point']

# b. Divers pour simple affichage
obsBrutesColsAff = ['Observateur', colPassage] + obsBrutesColsAff

# c. FieldDataSet
inventaireCols = ['Observateur', 'ID liste', 'Date', colPassage, 'Point']
obsIndivCols = ['Observateur', 'ID liste', 'Date', colPassage, 'Durée',
                'Ref', 'Point', 'Horaire', colEspece, 'Adulte', colDistance]

# e. Carto, rapports d'analyses ...
sousEtude = '-Nat'
titreEtude = 'ACDC 2019 Naturalist'
descrEtude = "Estimation des populations d'oiseaux du plateau de Cournols - Olloix - Montaigut-le-Blanc en 2019" \
             " par points d'écoute Distance Sampling sur smartphone (mêmes points que pour l'étude Papyrus)"
motsClesEtude = 'ACDC, Cournols, Olloix, 2019, DS, Naturalist, Smartphone'

[XII. Chargement des données prêtes pour l'analyse DS](#XII.-Chargement-des-donn%C3%A9es-pr%C3%AAtes-pour-l'analyse-DS)

# XII. Chargement des données prêtes pour l'analyse DS

Pour éviter de tout refaire, exécuter d'abord (et seulement) tout jusqu'à I. inclus, avant ce qui suit.

Attention: Dans ce cas, on observe parfois des différences (minimes) sur les distances observateur - oiseau,
           avec arrondi des 1, 2 ou 3 derniers chiffres significatifs (lors du passage dans Excel) ;
           ce qui correspond, pour des distances <= 1000m, à 10^-14m ... pas très grave ;-)
           
<p>N.B. A partir de la version 0.9.2 (22/01/2022), pyaudisam peut-être utilisé en ligne de commande pour rendre
   exactement les mêmes services que ceux des chapitres qui suivent (exemple: Cf. <a href="./donnees/acdc/acdc-2019-ds-params.py">acdc-2019-ds-params.py</a>).</p>

In [ ]:
fpn = dossier / f'{nomEtude}{sousEtude}-ObsIndivDist.xlsx'
with pd.ExcelFile(fpn) as xlsFile:
    dfObsCatIndiv = pd.read_excel(xlsFile, sheet_name='Donnees')
    dfTransects = pd.read_excel(xlsFile, sheet_name='Inventaires')

print(dict(etude=nomEtude+sousEtude, donnees=len(dfObsCatIndiv), inventaires=len(dfTransects)))

In [ ]:
dfTransects

In [ ]:
dfObsCatIndiv

[Annexe. Bilan & stats données individualisées](#Annexe.-Bilan-%26-stats-donn%C3%A9es-individualis%C3%A9es)

# XIII. Sélection des échantillons pour analyses DS

In [ ]:
# Colonnes de sélection des échantillons réellement présentes dans dfObsCatIndiv
samplingCols = [colEspece, colPassage] + colsSpeSelEchant
samplingCols = [col for col in samplingCols if col in dfObsCatIndiv.columns]

samplingCols

## 1. Examen des données

In [ ]:
dfObsCatIndiv.head()

In [ ]:
nonSpeciesSamplingCols = [col for col in samplingCols if col != colEspece]
nonSpeciesSamplingCols

In [ ]:
# Nombre d'individus par espèce, pour voir quelles espèces on va analyser
if groupage: # Clustering lors des analyses DS
    
    # Attention : Non testé encore !
    dfNObsCatIndiv = dfObsCatIndiv[samplingCols + ['Nombre']].groupby(samplingCols).sum()
    dfNObsCatIndiv.rename(columns=dict(Nombre='Individus'), inplace=True)
    
else: # Pas de clustering.
    
    dfNObsCatIndiv = dfObsCatIndiv[samplingCols + [colDistance]].groupby(samplingCols).count()
    dfNObsCatIndiv.rename(columns=dict(Distance='Individus'), inplace=True)

dfNObsCatIndiv.reset_index(inplace=True)

dfNObsCatIndiv.sort_values(by='Individus', ascending=False).head(15)

## 2. Choix des variantes

**Attention** : Variantes "Passage" : Présence obligatoire pour les pré-analyses et analyses, même si 1 seul passage !

In [ ]:
# Initialisation
variants = dict()  # Clef = nom des catégories

### Stats pour décider : Nombres d'individus contactés par espèce

In [ ]:
# Calcul des effectifs totaux.
dfNObsTotIndiv = dfNObsCatIndiv
if dCategoriesImbriqueesTout:  # Attention aux catégories imbriquées : données dupliquées exprès !
    for colCat, valTout in dCategoriesImbriqueesTout.items():
        dfNObsTotIndiv = dfNObsTotIndiv[dfNObsTotIndiv[colCat] == valTout]
dfNObsTotIndiv = dfNObsTotIndiv[[colEspece, 'Individus']].groupby(colEspece).sum() \
                    .sort_values(by='Individus', ascending=False)

dfNObsTotIndiv.head()

In [ ]:
dfNObsTotIndiv[dfNObsTotIndiv.Individus >= 60].index

### Ou : Les espèces avec au moins N individus contactés

In [ ]:
# Les espèces avec un minimum d'individus observés ...
nMinTotIndivs = 20

In [ ]:
variants[colEspece] = list(dfNObsTotIndiv[dfNObsTotIndiv.Individus >= nMinTotIndivs].index)
print(len(variants[colEspece]), ', '.join(variants[colEspece]))

variants['Adulte'] = ['m', 'm+a']

variants[colPassage] = [''] # Tous les passages ensemble

### Ou : Les N espèces les plus notées

In [ ]:
# Les N les plus notées ...
nPlusTotIndivs = 22

variants[colEspece] = list(dfNObsTotIndiv.index[:nPlusTotIndivs])
print(len(variants[colEspece]), ', '.join(variants[colEspece]))

### Ou : ACDC 2019

In [ ]:
# Les espèces avec au moins 27 mâles contactés dans les 10mn
#nMinMal10 = 27 # Comme ACDC 2019 Papyrus
#
#dfNObsMal10 = dfNObsCatIndiv[dfNObsCatIndiv.Adulte == 'm']
#if dCategoriesImbriqueesTout:  # Attention aux catégories imbriquées : données dupliquées exprès !
#    for colCat, valTout in dCategoriesImbriqueesTout.items():
#        dfNObsMal10 = dfNObsMal10[dfNObsMal10[colCat] == valTout]
#
#dfNObsMal10 = dfNObsMal10[[colEspece, 'Individus']].groupby(colEspece).sum() \
#                  .sort_values(by='Individus', ascending=False)
#
#variants[colEspece] = list(dfNObsMal10[dfNObsMal10.Individus >= nMinMal10].index)
#print(', '.join(variants[colEspece]), '=>', len(variants[colEspece]), 'espèces')
#
#dfNObsMal10.head(35)

In [ ]:
# Explicitement : ... en gros les 30 espèces les plus notées et analysables en DS
variants[colEspece] = ['Sylvia atricapilla', 'Turdus merula', 'Alauda arvensis', 'Sylvia communis',
                       'Columba palumbus', 'Luscinia megarhynchos', 'Cuculus canorus', 'Fringilla coelebs',
                       'Phylloscopus collybita', 'Parus major', 'Lullula arborea', 'Emberiza cirlus',
                       'Erithacus rubecula', 'Saxicola rubicola', 'Emberiza citrinella', 'Cyanistes caeruleus',
                       'Turdus philomelos', 'Emberiza calandra', 'Turdus viscivorus', 'Prunella modularis',
                       'Phylloscopus bonelli', 'Carduelis cannabina', 'Jynx torquilla', 'Carduelis chloris',
                       'Anthus trivialis', 'Upupa epops', 'Lanius collurio', 'Oriolus oriolus',
                       'Streptopelia turtur', 'Dendrocopos major']
print(len(variants[colEspece]), 'espèces =>', ', '.join(variants[colEspece]))

variants[colPassage] = ['b', 'a+b'] # Passage b ou a+b => 2 variantes

variants['Durée'] = ['5mn', '10mn'] # 5 1ères mn, ou toutes les 10 => 2 variantes

variants['Adulte'] = ['m', 'm+a'] # Les mâles, et ensuite les mâles et autres adultes (=> 2 variantes)

In [ ]:
# Pour info ... et action ? Liste complémentaire (à + de 20 individus) pour Nat+Pap
compltNatPap = ['Corvus corone', 'Garrulus glandarius', 'Troglodytes troglodytes',  'Pica pica',
                'Picus viridis', 'Hippolais polyglotta', 'Carduelis carduelis', 'Certhia brachydactyla',
                'Sylvia borin', 'Coturnix coturnix']
exclusNatPap = ['Sturnus vulgaris', 'Streptopelia decaocto', 'Sturnus vulgaris', 'Streptopelia decaocto', ]

# N.B. Non encore utilisé ...

## 3. Spec. implicite des variantes

=> combinaisons à générer automatiquement

In [ ]:
dImplSampleSpecs = variants
dImplSampleSpecs

In [ ]:
# Utilitaire : génération du fichier de specs (implicite) correspondant
# nSampSpecRows = max(len(vals) for vals in dImplSampleSpecs.values())
# dfImplSampSpecs = pd.DataFrame({col: vals + [np.nan] * (nSampSpecRows - len(vals)) for col, vals in dImplSampleSpecs.items()})
# dfImplSampSpecs.loc[0, 'Commentaire'] = 'Toutes combinaisons de ces 4 colonnes'
# fpn = dossier / f'{nomEtude}-Echantillons.xlsx'
# dfImplSampSpecs.to_excel(fpn, index=False, sheet_name='Echantillons_impl')

## 4. Pour info., qq stats sur les échantillons ainsi spécifiés

In [ ]:
# Les specs explicitées donneront ...
dfExplSampleSpecs = ads.DSAnalyser.explicitVariantSpecs(dict(_impl=dImplSampleSpecs))
dfExplSampleSpecs

In [ ]:
# Distance max et nombre d'individus pour chaque échantillon, dans l'ordre d'analyse (Cf. XIII)
# a. Colonnes réelles de sélection des échantillons (certaines peuvent être vides)
dfSampleOrd = dfExplSampleSpecs.dropna(axis='columns', how='all')
indexCols = [col for col in samplingCols if col in dfSampleOrd.columns]

# b. Ordre des espèces dans les échantillons
dfSampleOrd = dfSampleOrd.drop_duplicates(subset=[colEspece])[[colEspece]].reset_index(drop=True)
dfSampleOrd = dfSampleOrd.reset_index(drop=False).rename(columns=dict(index='order'))
dfSampleOrd = dfSampleOrd.set_index(colEspece)

# c. Distances max pour les valeurs uniques des catégories (ex: Adulte => m, a)
dfSampleStats = dfObsCatIndiv[indexCols + [colDistance]].groupby(indexCols).agg(['min', 'max', 'count'])
dfSampleStats.columns = ['Distance Min', 'Distance Max', 'NTot Obs']
dfSampleStats = dfSampleStats.reset_index()

# d. Categories pour combinaisons + tri "simple" dans le bon ordre
speciesOrder = list(dfSampleOrd.index) + [e for e in dfSampleStats[colEspece].unique() if e not in dfSampleOrd.index]

dCategories = {colEspece: speciesOrder, colPassage: ['a', 'b', 'a+b'],
               'Adulte': ['m', 'a', 'm+a'], 'Durée': ['5mn', '10mn']}
dCategoryTypes = { cat: pdt.CategoricalDtype(categories=values, ordered=True) for cat, values in dCategories.items()}

for col in indexCols:
    dfSampleStats[col] = dfSampleStats[col].astype(dCategoryTypes[col])

# e. Distances max pour les valeurs combinées des catégories non imbriquées (ex: Adulte => m+a)
cols2OrCombine = [col for col in nonSpeciesSamplingCols if col in indexCols and col not in dCategoriesImbriquees.keys()]
for col2OrComb in cols2OrCombine:
    indexNoCol2CombCols = [col for col in indexCols if col != col2OrComb]
    dfSampleStatsOrComb = \
        dfSampleStats[indexNoCol2CombCols + ['Distance Min', 'Distance Max', 'NTot Obs']].groupby(indexNoCol2CombCols) \
            .agg({'Distance Min': 'min', 'Distance Max': 'max', 'NTot Obs': 'sum'})
    dfSampleStatsOrComb.columns = ['Distance Min', 'Distance Max', 'NTot Obs']
    dfSampleStatsOrComb = dfSampleStatsOrComb.dropna().reset_index()  # Why Nans appear in index ? A mystery !
    dfSampleStatsOrComb[col2OrComb] = '+'.join(dfSampleStats[col2OrComb].sort_values().unique())
    dfSampleStatsOrComb[col2OrComb] = dfSampleStatsOrComb[col2OrComb].astype(dCategoryTypes[col2OrComb])
    dfSampleStats = dfSampleStats.append(dfSampleStatsOrComb, ignore_index=True)
    
# d. Tri dans l'ordre des espèces, et des autres colonnes de sélection d'échantillon    
dfSampleStats.sort_values(by=indexCols, inplace=True)  # Magic ! (thanks to CategoricalDtype)

dfSampleStats.reset_index(inplace=True, drop=True)

dfSampleStats

In [ ]:
# Sauvegarde.
fpn = dossier / f'{nomEtude}{sousEtude}-StatsEchantillons.xlsx'

dfSampleStats.to_excel(fpn, index=False)

logger.info(fpn.as_posix())

# XIV. Export pour analyses "manuelles" dans Distance

(exécuter d'abord jusqu'à I ; puis II à XI (sauf IX), ou alors juste XIII et XIV)

## 1. Paramètres pour l'export et les (pré-)analyses

In [ ]:
# Description des données
transectPlaceCols = ['Point']
passIdCol = colPassage

effortCol = colEffort
sampleDistCol = colDistance
sampleDecCols = [effortCol, sampleDistCol]

sampleNumCol = 'Echant'
sampleSelCols = [colEspece, passIdCol] + colsSpeSelEchant

sampleAbbrevCol = 'Abrev. Echant'

In [ ]:
# Chaîne courte d'identification d'une spec. d'échantillon.
def sampleAbbrev(sSamp):
    abbrvs = [''.join(word[:4].title() for word in sSamp[colEspece].split(' ')[:2])]
    if colPassage in sSamp.index and not pd.isnull(sSamp.Passage) and sSamp.Passage:
        abbrvs.append(sSamp.Passage.replace('+', ''))
    if 'Durée' in sSamp.index:
        abbrvs.append(sSamp['Durée'].replace('+', ''))
    if 'Adulte' in sSamp.index:
        abbrvs.append(sSamp.Adulte.replace('+', ''))
    return '-'.join(abbrvs)

[XV. Pré-analyses automatiques](#XV.-Pr%C3%A9-analyses-automatiques)

[XVI. Analyses automatiques](#XVI.-Analyses-automatiques)

## 2. Export Distance

In [ ]:
# Dossier de sortie : résultats, rapports ... etc.
workDir = dossier / dt.datetime.now().strftime('%y%m%d-%H%M')
workDir.as_posix()

In [ ]:
# Objet PreAnalyser pour l'export.
pranlysr = ads.MCDSPreAnalyser(dfObsCatIndiv, dfTransects=dfTransects, effortConstVal=effortConst, dSurveyArea=dZoneEtude, 
                               transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                               sampleSelCols=sampleSelCols, sampleDecCols=sampleDecCols, sampleIndCol=sampleNumCol,
                               abbrevCol=sampleAbbrevCol, abbrevBuilder=sampleAbbrev,
                               distanceUnit=distanceUnit, areaUnit=areaUnit,
                               surveyType=surveyType, distanceType=distanceType, clustering=groupage,
                               workDir=workDir)

In [ ]:
# Export au format Distance des échantillons sélectionnés.
pranlysr.exportDSInputData(implSampleSpecs=dict(_impl=dImplSampleSpecs))

# XV. Pré-analyses automatiques

(exécuter d'abord jusqu'à I ; puis II à XI (sauf IX), ou alors juste XIII et XIV)

## 1. Paramètres de pré-analyse

In [ ]:
# Exécuter XIV.1

[XIV.1. Paramètres pour l'export et les (pré-)analyses](#1.-Param%C3%A8tres-pour-l'export-et-les-(pr%C3%A9-)analyses)

## 2a. Ou: Exécution des pré-analyses

In [ ]:
# Dossier de sortie : résultats, rapports ... etc.
workDir = dossier / dt.datetime.now().strftime('%y%m%d-%H%M')

presFileName = workDir / f'{nomEtude}{sousEtude}-PreAnalyses-resultats.xlsx'

presFileName.as_posix()

In [ ]:
# Objet PréAnalyser.
pranlysr = ads.MCDSPreAnalyser(dfObsCatIndiv, dfTransects=dfTransects, effortConstVal=effortConst, dSurveyArea=dZoneEtude, 
                               transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                               sampleSelCols=sampleSelCols, sampleDecCols=sampleDecCols, sampleIndCol=sampleNumCol,
                               abbrevCol=sampleAbbrevCol, abbrevBuilder=sampleAbbrev,
                               distanceUnit=distanceUnit, areaUnit=areaUnit,
                               surveyType=surveyType, distanceType=distanceType, clustering=groupage,
                               resultsHeadCols=dict(before=[sampleNumCol], sample=sampleSelCols, after=[sampleAbbrevCol]),
                               workDir=workDir)

In [ ]:
# Vérification des specs de pré-analyse (échantillons)
dfExplSampleSpecs, userParamSpecCols, intParamSpecCols, unmUserParamSpecCols, verdict, reasons = \
    pranlysr.explicitParamSpecs(implParamSpecs=dict(_impl=dImplSampleSpecs), dropDupes=True, check=True)  

logger.info(dict(nSamples=len(dfExplSampleSpecs)))

assert userParamSpecCols == [] # No analysis params here (auto. generated by PreAnalyser)
assert intParamSpecCols == [] # Idem
assert verdict
assert not reasons

In [ ]:
# Stratégie choix modèles.
lModelStrategy = [dict(keyFn=kf, adjSr=js, estCrit='AIC', cvInt=95)
                  for js in['COSINE', 'POLY']  #, 'HERMITE'] # Hermite : allonge les calculs, pour des bricoles (?)
                  for kf in['HNORMAL', 'HAZARD', 'UNIFORM']]  #, 'NEXPON']] # NegExpon : Pb car g'(0) << 0 !!!
pd.DataFrame(lModelStrategy)

In [ ]:
%%time

preResults = pranlysr.run(implSampleSpecs=dict(_impl=dImplSampleSpecs), dModelStrategy=lModelStrategy, threads=12)

preAnalysed = True

pranlysr.shutdown()

# Qq specs supplémentaires
if 'dfSampleStats' in dir():
    preResults.updateSpecs(sampleStats=dfSampleStats)

# Sauvegarde résultats
preResults.toExcel(presFileName)

backup(presFileName)

Performances figures on a 4-core HT i5-8350U Ruindows 10 laptop with PCI-e SSD,
"optimal performance power scheme", 12 threads, Python 3.8 :
* ACDC2019 3mn40s à 3mn54s 240 analyses, 12 modèles
* ACDC2019 Pap 2mn09s 240 analyses, 6 modèles

## 2b. Ou: Chargement des résultats des pré-analyses déjà faites

In [ ]:
if 'preAnalysed' not in dir():
    preAnalysed = False

In [ ]:
if not preAnalysed:
    
    resFolders = [fn.name for fn in dossier.glob('[0-9]'*6+'-'+'[0-9]'*4)
                  if (fn / f'{nomEtude}{sousEtude}-PreAnalyses-resultats.xlsx').is_file()]
    
    logger.info('Résultats disponibles : ' + ', '.join(resFolders))

In [ ]:
if not preAnalysed:
    
    workDir = dossier / resFolders[1]  # <=== Choisir le dossier de résultats ici.
    
    presFileName = workDir / f'{nomEtude}{sousEtude}-PreAnalyses-resultats.xlsx'
    
    logger.info(f'Fichier choisi : {presFileName}')

In [ ]:
if not preAnalysed:
    
    # An analyser object knowns how to build an empty results object ...
    panlysr = ads.MCDSPreAnalyser(dfObsCatIndiv, dfTransects=dfTransects, effortConstVal=effortConst, dSurveyArea=dZoneEtude, 
                                  transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                                  sampleSelCols=sampleSelCols, sampleDecCols=sampleDecCols, sampleIndCol=sampleNumCol,
                                  abbrevCol=sampleAbbrevCol, abbrevBuilder=sampleAbbrev,
                                  distanceUnit=distanceUnit, areaUnit=areaUnit,
                                  surveyType=surveyType, distanceType=distanceType, clustering=groupage,
                                  resultsHeadCols=dict(before=[sampleNumCol], sample=sampleSelCols, after=[sampleAbbrevCol]))
    
    preResults = panlysr.setupResults()
    
    # Load results from file
    preResults.fromFile(presFileName)
    
else:
    
    logger.info('Déjà calculé, pas de rechargement ...')
    
logger.info('... {} pré-analyses prêtes pour un rapport'.format(len(preResults)))

## 3. Rapports Excel et HTML

In [ ]:
#preResults._dfData = preResults._dfData.iloc[:8]

In [ ]:
preResults.dfData.head()

In [ ]:
# Sélection des colonnes pour les tableaux du pré-rapport
# a. Page principale : Colonne 1 (haut), de description de l'échantillon
preRepSampleCols = [('header (sample)', col, 'Value') for col in samplingCols]

# b. Page principale : Colonne 1 (bas), des paramètres du modèle d'analyse
preRepParamCols = \
[
    ('parameters', 'estimator key function', 'Value'),
    ('parameters', 'estimator adjustment series', 'Value'),
    ('parameters', 'CV interval', 'Value')
]

# c. Page principale : Colonne 2 et 3, des résultats (juste avant les 4, 5, et 6 avec les courbes)
preRepResultCols = \
[
    ('run output', 'run status', 'Value'),
    
    ('encounter rate', 'number of observations (n)', 'Value'),
    ('encounter rate', 'right truncation distance (w)', 'Value'),
    ('encounter rate', 'effort (L or K or T)', 'Value'),
    
    ('detection probability', 'AIC value', 'Value'),
    ('detection probability', 'chi-square test probability determined', 'Value'),
    ('detection probability', 'Kolmogorov-Smirnov test probability', 'Value'),
    ('detection probability', 'probability of detection (Pw)', 'Value'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Value'),

    ('density/abundance', 'density of animals', 'Cv'),
    ('density/abundance', 'density of animals', 'Value'),
    ('density/abundance', 'density of animals', 'Lcl'),
    ('density/abundance', 'density of animals', 'Ucl'),
    
    ('density/abundance', 'number of animals, if survey area is specified', 'Value'),
    ('density/abundance', 'number of animals, if survey area is specified', 'Lcl'),
    ('density/abundance', 'number of animals, if survey area is specified', 'Ucl')
   
]

# d. Pages ppale et de détails : Tableau de synthèse.
preRepSynthCols = preRepSampleCols + preRepParamCols \
+ [
    ('run output', 'run status', 'Value'),
    
    ('encounter rate', 'number of observations (n)', 'Value'),
    ('encounter rate', 'right truncation distance (w)', 'Value'),
    ('encounter rate', 'effort (L or K or T)', 'Value'),
    
    ('detection probability', 'AIC value', 'Value'),
    ('detection probability', 'chi-square test probability determined', 'Value'),
    ('detection probability', 'Kolmogorov-Smirnov test probability', 'Value'),
    ('density/abundance', 'density of animals', 'Cv'),
    
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Value'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Lcl'),
    ('detection probability', 'effective strip width (ESW) or effective detection radius (EDR)', 'Ucl'),
    
    ('density/abundance', 'density of animals', 'Value'),
    ('density/abundance', 'density of animals', 'Lcl'),
    ('density/abundance', 'density of animals', 'Ucl'),
    
    ('detection probability', 'probability of detection (Pw)', 'Value'),
    ('detection probability', 'probability of detection (Pw)', 'Lcl'),
    ('detection probability', 'probability of detection (Pw)', 'Ucl'),
    ('detection probability', 'probability of detection (Pw)', 'Df'),

    ('density/abundance', 'number of animals, if survey area is specified', 'Value'),
    ('density/abundance', 'number of animals, if survey area is specified', 'Lcl'),
    ('density/abundance', 'number of animals, if survey area is specified', 'Ucl'),
    ('density/abundance', 'number of animals, if survey area is specified', 'Df'),
   
    ('run output', 'run folder', 'Value')
]

In [ ]:
preReport = ads.MCDSResultsPreReport(resultsSet=preResults,
                                     title=titreEtude, subTitle='Rapport de pré-analyse',
                                     anlysSubTitle='Détail des pré-analyses', description=descrEtude,
                                     keywords=motsClesEtude, pySources=['Visionature-ds-points.ipynb'],
                                     lang='fr', #plotImgSize=(640, 400), superSynthPlotsHeight=288,
                                     #plotImgQuality=80, plotImgFormat='jpg', # Same final size as raw PNG :-(
                                     sampleCols=preRepSampleCols, paramCols=preRepParamCols,
                                     resultCols=preRepResultCols, synthCols=preRepSynthCols,
                                     tgtFolder=workDir, tgtPrefix=f'{nomEtude}{sousEtude}-PreAnalyses-rapport')

In [ ]:
xlsxPreRep = preReport.toExcel()

backup(xlsxPreRep)

HTML(f'Rapport Excel : <a href="{xlsxPreRep}" target="blank">{xlsxPreRep}</a>')

In [ ]:
%%time

htmlPreRep = preReport.toHtml(generators=6)

backup(htmlPreRep)

HTML(f'Pré-rapport HTML : <a href="{htmlPreRep}" target="blank">{htmlPreRep}</a>')

Performances figures on a 4-core HT i5-8350U Ruindows 10 laptop with PCI-e SSD,
"optimal performance power scheme", 12 threads, Python 3.8 :
* CretesPlombCantalZPS2020 : 55s for 1 generator, >= 28s for 4 or 6 or 8 generators (but old).
* ACDC2019 240 samples (6 generators) : 2mn30s, then 3mn50s (why ?), then 4mn30s (added fixed bin hist)
  then 5mn30s (3 superimposed fixed bin hists).


In [ ]:
# Nouveauté Pandas 1.1, mais hélas, pas de colorisation via style en 1.1.3 :-(
#odsPreRep = preReport.toOpenDoc()

#HTML(f'Rapport OpenDoc/Spreadsheet : <a href="{odsPreRep}" target="blank">{odsPreRep}</a>')

# XVI. Analyses automatiques

(avec optimisation de troncatures éventuelles)

In [ ]:
# Chaîne courte d'identification d'une analyse.
def analysisAbbrev(sAnlys):
    
    # Sample abbreviation
    abbrevs = [sampleAbbrev(sAnlys)]

    # Model + Parameters abbreviation
    abbrevs += [sAnlys['FonctionClé'][:3].lower(), sAnlys['SérieAjust'][:3].lower()]
    dTroncAbbrv = { 'l': 'TrGche' if 'TrGche' in sAnlys.index else 'TroncGche',
                    'r': 'TrDrte' if 'TrDrte' in sAnlys.index else 'TroncDrte',
                    'm': 'NbTrModel' if 'NbTrModel' in sAnlys.index else  'NbTrchMod',
                    'd': 'NbTrDiscr' }
    for abbrev, name in dTroncAbbrv.items():
        if name in sAnlys.index and not pd.isnull(sAnlys[name]):
            abbrevs.append('{}{}'.format(abbrev, sAnlys[name][0].lower() if isinstance(sAnlys[name], str)
                                                 else int(sAnlys[name])))
   
    return '-'.join(abbrevs)

In [ ]:
optAnalysed = False

## 1. Paramètres d'optanalyse

(exécuter d'abord jusqu'à I ; puis II à X, ou alors juste XII ; puis XIV.1)

### a. Description des données

In [ ]:
# 1. Cf. XIV.1 pour les paramètres utiles pour l'export et les pré-analyses

# 2. Compléments pour les optanalyses.
anlysIndCol = 'Analyse'
anlysAbbrevCol = 'Abrev. Analyse'
anlysParamCols = ['FonctionClé', 'SérieAjust', 'TrGche', 'TrDrte', 'NbTrchMod']

### b. Paramètres d'optimisation par défaut

In [ ]:
defEstimKeyFn = 'HNORMAL'
defEstimAdjustFn = 'COSINE'
defEstimCriterion = 'AIC'
defCVInterval = 95

defMinDist = None
defMaxDist = None, 
defFitDistCuts = None
defDiscrDistCuts = None

defExpr2Optimise = 'chi2'
defMinimiseExpr = False
defOutliersMethod = 'tucquant'
defOutliersQuantCutPct = 5
defFitDistCutsFctr = dict(min=2/3, max=3/2)
defDiscrDistCutsFctr = dict(min=1/3, max=1)

defSubmitTimes = 2
defSubmitOnlyBest = 1
dDefSubmitOtherParams = dict()

defCoreEngine = 'zoopt'
defCoreMaxIters = 200
defCoreTermExprValue = None
defCoreAlgorithm = 'racos'
defCoreMaxRetries = 0

### c. Paramètres pour les post-calculs

In [ ]:
ldTruncIntrvSpecs = [dict(col='left', minDist=5.0, maxLen=5.0),
                     dict(col='right', minDist=25.0, maxLen=25.0)]
truncIntrvEpsilon = 1e-6

### d. Analyses à faire : variante d'études

(sources = fichiers Excel / ODF de specs d'opt-analyse)

In [ ]:
nomFicSpecs = f'{nomEtude}-OptAnalysesAFaire'
ignorerSpecs = []

#### Ou : Qq soit l'étude : la totale

In [ ]:
varEtude = ''

#### Ou : ACDC 2019, sans les analyses optimisées

In [ ]:
varEtude = 'SansOptim'
ignorerSpecs = ['TroncaturesAuto_impl']

#### Ou : ACDC 2019, avec les analyses optimisées, mais sans refaire les optimisations

(recharger les résultats au préalable, via [1b. Ou: Chargement des résultats des opt-analyses](#1b.-Ou%3A-Chargement-des-r%C3%A9sultats-des-opt-analyses) ci-dessous)

In [ ]:
varEtude = 'SansReOptim'
nomFicSpecs = None

In [ ]:
dfExplOptAnlysSpecs = results.dfTransData('fr')[['Espèce', 'Passage', 'Adulte', 'Durée', 'FonctionClé', 'SérieAjust',
                                                 'TrGche', 'TrDrte', 'NbTrchMod', 'OptimTrunc']]
dfExplOptAnlysSpecs.head()

#### Ou : ZPS Cantal 2020 : pour comparer aux analyses manuelles de Mathis

In [ ]:
varEtude = 'CommeMathis'
nomFicSpecs = f'{nomEtude}-OptAnalysesAFaire-{varEtude}'

### e. Analyses à faire : fichiers de specs, ou specs explicites.

In [ ]:
dict(etude=varEtude, specs=nomFicSpecs if nomFicSpecs else 'dfExplOptAnlysSpecs', ignorer=ignorerSpecs)

In [ ]:
# Les analyses à faire (avec specs d'optimisation dedans si nécessaire)
if nomFicSpecs:
    
    dfExplOptAnlysSpecs = None
    
    optAnlysSpecFileExts = ['.ods', '.xlsx']
    for ext in optAnlysSpecFileExts:
        optAnlysSpecs = dossier / f'{nomFicSpecs}{ext}'
        if optAnlysSpecs.is_file():
            break

    assert optAnlysSpecs.is_file(), \
           '{} n\'existe pas, ni les autres extensions possibles [{}] !' \
           .format(optAnlysSpecs.as_posix(), ', '.join(optAnlysSpecFileExts[:-1]))

    logger.info('Implicites, via ' + optAnlysSpecs.as_posix())

    if ignorerSpecs:
        optAnlysSpecs = pd.read_excel(optAnlysSpecs, sheet_name=None)
        for spec in ignorerSpecs:
            del optAnlysSpecs[spec]
        logger.info('Retiré ' + str(ignorerSpecs))
        logger.info('Restent ' + str(list(optAnlysSpecs.keys())))
        
else:
    
    optAnlysSpecs = None
    
    assert not dfExplOptAnlysSpecs.empty
    
    logger.info('Explicites, via dfExplOptAnlysSpecs')

## 2a. Ou: Exécution des opt-analyses

In [ ]:
# Set to True for recovering when interrupted during optimisations.
recoverOptims = False

In [ ]:
# List possible folders for recovery.
if recoverOptims:
    
    logger.info('Dossiers possibles pour reprise :')
    
    bkupFileNamePat = 'optr-resbak-[01].pickle.xz'
    resFolders = list()
    folderInd = 0
    for fpn in sorted(dossier.glob('[0-9]'*6+'-'+'[0-9]'*4)):
        
        bkupFilePathNames = list(fpn.glob(bkupFileNamePat))
        if bkupFilePathNames:
            
            logger.info(f'  #{folderInd} {fpn.name}')
            for bfpn in bkupFilePathNames:
                logger.info('    {} {}'.format(bfpn.name, pd.Timestamp.fromtimestamp(bfpn.stat().st_mtime)))
            
            resFolders.append(fpn.name)
            folderInd += 1

In [ ]:
# Select run folder : for recovery or a brand new one.
if recoverOptims:
    
    # Choose manually the folder to recover from / go on working inside.
    resFolder = resFolders[1]  # <===== Here, set the chosen index
    
else:
    
    # A brand new one.
    resFolder = dt.datetime.now().strftime('%y%m%d-%H%M')
    
# Dossier de sortie : résultats, rapports ... etc.
workDir = dossier / resFolder

workDir.as_posix()

In [ ]:
# Fichier de résultats
resFileName = workDir / f'{nomEtude}{sousEtude}-OptAnalyses{varEtude}-resultats.xlsx'

resFileName.as_posix()

In [ ]:
optanlr = \
    ads.MCDSTruncationOptanalyser(dfObsCatIndiv, dfTransects=dfTransects, effortConstVal=effortConst, dSurveyArea=dZoneEtude, 
                                  transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                                  sampleSelCols=sampleSelCols, sampleDecCols=sampleDecCols, sampleDistCol=sampleDistCol,
                                  abbrevCol=anlysAbbrevCol, abbrevBuilder=analysisAbbrev,
                                  anlysIndCol=anlysIndCol, sampleIndCol=sampleNumCol,
                                  distanceUnit=distanceUnit, areaUnit=areaUnit,
                                  surveyType=surveyType, distanceType=distanceType, clustering=groupage,
                                  resultsHeadCols=dict(before=[anlysIndCol, sampleNumCol], sample=sampleSelCols,
                                                       after=anlysParamCols + [anlysAbbrevCol]),
                                  ldTruncIntrvSpecs=ldTruncIntrvSpecs, truncIntrvEpsilon=truncIntrvEpsilon,
                                  workDir=workDir, runMethod='subprocess.run', runTimeOut=300,
                                  defEstimKeyFn=defEstimKeyFn, defEstimAdjustFn=defEstimAdjustFn,
                                  defEstimCriterion=defEstimCriterion, defCVInterval=defCVInterval,
                                  defExpr2Optimise=defExpr2Optimise, defMinimiseExpr=defMinimiseExpr,
                                  defOutliersMethod=defOutliersMethod, defOutliersQuantCutPct=defOutliersQuantCutPct,
                                  defFitDistCutsFctr=defFitDistCutsFctr, defDiscrDistCutsFctr=defDiscrDistCutsFctr,
                                  defSubmitTimes=defSubmitTimes, defSubmitOnlyBest=defSubmitOnlyBest,
                                  dDefSubmitOtherParams=dDefSubmitOtherParams,
                                  dDefOptimCoreParams=dict(core=defCoreEngine, maxIters=defCoreMaxIters,
                                                           termExprValue=defCoreTermExprValue,
                                                           algorithm=defCoreAlgorithm, maxRetries=defCoreMaxRetries))

In [ ]:
dfExplOptAnlysSpecs

In [ ]:
# Vérification des specs d'opt-analyse
dfExplOptAnlysSpecs, userParamSpecCols, intParamSpecCols, unmUserParamSpecCols, verdict, reasons = \
    optanlr.explicitParamSpecs(implParamSpecs=optAnlysSpecs, dfExplParamSpecs=dfExplOptAnlysSpecs, dropDupes=True, check=True)  

if not verdict:
    logger.info('Optanalysis specs errors:')
    logger.info('\n'.join(reasons))
else:
    logger.info('Optanalysis specs OK')

logger.info(dict(specs=', '.join(optAnlysSpecs.keys()) if isinstance(optAnlysSpecs, dict)
                       else optAnlysSpecs.as_posix() if optAnlysSpecs else 'Explicit',
                 nOptAnalyses=len(dfExplOptAnlysSpecs), userParamSpecCols=', '.join(userParamSpecCols),
                 intParamSpecCols=', '.join(intParamSpecCols), unmUserParamSpecCols=', '.join(unmUserParamSpecCols)))

assert verdict
assert not reasons

In [ ]:
# Les voici explicitées (juste pour voir ... à moins que ...)
dfExplOptAnlysSpecs.to_excel(dossier / f'{nomEtude}{sousEtude}-ExplOptAnlysSpecs.xlsx')

In [ ]:
# Vérification que les échantillons concernés ont été pré-analysés.
if 'dfExplSampleSpecs' in dir():
    
    dfOptAnlysSpecsCheck = dfExplOptAnlysSpecs[sampleSelCols].drop_duplicates()
    dfOptAnlysSpecsCheck['OptAnalyses'] = True
    dfOptAnlysSpecsCheck.set_index(sampleSelCols, inplace=True)

    dfPreAnlysSpecsCheck = dfExplSampleSpecs.copy()
    dfPreAnlysSpecsCheck['PreAnalyses'] = True
    dfPreAnlysSpecsCheck.set_index(sampleSelCols, inplace=True)
    
    dfCheck = dfOptAnlysSpecsCheck.join(dfPreAnlysSpecsCheck, how='outer').reset_index()
    
else:
    
    dfCheck = pd.DataFrame(columns=['PreAnalyses', 'OptAnalyses'])
    
logger.info(f'{len(dfCheck} échantillons pré-analysés ou à analyser, au total.')
logger.info('Voici ceux qui n\'ont pas été préanalysés :')
dfCheck[dfCheck.PreAnalyses.isnull()]

In [ ]:
# Comment-out to force using explicit specs.
#optAnlysSpecs = None

# Use implicit specs if given (not deduced explicit ones).
if optAnlysSpecs:
    dfExplOptAnlysSpecs = None
    
# Last checks.
dict(recoverOptims=recoverOptims, dfExplOptAnlysSpecs=len([] if dfExplOptAnlysSpecs is None else dfExplOptAnlysSpecs),
     optAnlysSpecs=optAnlysSpecs.as_posix() if optAnlysSpecs else None, workDir=workDir.as_posix())

In [ ]:
%%time

# Exécution des (opt-)analyses.
results = optanlr.run(dfExplParamSpecs=dfExplOptAnlysSpecs, implParamSpecs=optAnlysSpecs,
                      recoverOptims=recoverOptims, threads=12)

#results = optanlr.run(dfExplOptAnlysSpecs.iloc[0:2], threads=2)

optAnalysed = True

optanlr.shutdown()

# Qq specs supplémentaires
if 'dfSampleStats' in dir():
    results.updateSpecs(sampleStats=dfSampleStats)

# Sauvegarde résultats
results.toExcel(resFileName)

backup(resFileName)

In [ ]:
#optanlr.shutdown()

In [ ]:
results.dfData.head()

## 2b. Ou: Chargement des résultats des opt-analyses

(choisir le dossier parmis ceux qui sont présents)

In [ ]:
if 'optAnalysed' not in dir():
    optAnalysed = False
    
if not optAnalysed:
    
    resFolders = [fn.name for fn in dossier.glob('[0-9]'*6+'-'+'[0-9]'*4)
                  if (fn / f'{nomEtude}{sousEtude}-OptAnalyses{varEtude}-resultats.xlsx').is_file()]
    
    logger.info('Résultats disponibles : ' + ', '.join(resFolders))

In [ ]:
if not optAnalysed:
    
    workDir = dossier / resFolders[0]  # <=== Choisir le dossier de résultats ici.
    
    resFileName = workDir / f'{nomEtude}{sousEtude}-OptAnalyses{varEtude}-resultats.xlsx'
    
    logger.info(f'Fichier choisi : {resFileName.as_posix()}')

In [ ]:
if not optAnalysed:
    
    # An optanalyser object knowns how to build an empty results object ...
    optanlr = \
        ads.MCDSTruncationOptanalyser(dfObsCatIndiv, dfTransects=dfTransects,
                                      effortConstVal=effortConst, dSurveyArea=dZoneEtude, 
                                      transectPlaceCols=transectPlaceCols, passIdCol=passIdCol, effortCol=effortCol,
                                      sampleSelCols=sampleSelCols, sampleDecCols=sampleDecCols, sampleDistCol=sampleDistCol,
                                      abbrevCol=anlysAbbrevCol, abbrevBuilder=analysisAbbrev,
                                      anlysIndCol=anlysIndCol, sampleIndCol=sampleNumCol,
                                      distanceUnit=distanceUnit, areaUnit=areaUnit,
                                      surveyType=surveyType, distanceType=distanceType, clustering=groupage,
                                      ldTruncIntrvSpecs=ldTruncIntrvSpecs, truncIntrvEpsilon=truncIntrvEpsilon,
                                      resultsHeadCols=dict(before=[anlysIndCol, sampleNumCol], sample=sampleSelCols,
                                                           #after=anlysParamCols + [optimTruncCol, anlysAbbrevCol]))
                                                           after=anlysParamCols + [anlysAbbrevCol])) # TODO: test !
    
    results = optanlr.setupResults()
    
    # Load results from file
    results.fromFile(resFileName)
    
else:
    
    logger.info('Déjà calculé, pas de rechargement ...')
    
logger.info('... {} opt-analyses pour rapport'.format(len(results)))

In [ ]:
# Compatibilité ascendante pour résultats d'avant fin août 2021 : ajout des stats échantillons.
if not optAnalysed and any(col not in results._dfData.columns for col in ads.MCDSEngine.MIStatSampCols):
    
    # Add sample stats a posteriori (these stats had not been implemented when the historical results were saved to disk)
    #if 'dfSampleStats' not in dir():  # TODO: make this work is dfSampleStats already defined
    dfSampleStats = pd.read_excel(dossier / f'{nomEtude}{sousEtude}-StatsEchantillons.xlsx')
    dfSampleStats.rename(columns={'NTot Obs': 'NTot Obs0'}, inplace=True)
    dfSampleStats.insert(dfSampleStats.columns.to_list().index('Distance Min'), 'NTot Obs', dfSampleStats['NTot Obs0'])
    dfSampleStats.drop(columns=['NTot Obs0'], inplace=True)

    miSampleCols = pd.MultiIndex.from_tuples([('header (sample)', colEspece, 'Value'),
                                              ('header (sample)', colPassage, 'Value'),
                                              ('header (sample)', colsSpeSelEchant[0], 'Value'),
                                              ('header (sample)', colsSpeSelEchant[1], 'Value')])
    dfSampleStats.columns = miSampleCols.append(ads.MCDSEngine.MIStatSampCols)

    results.setData(results._dfData.join(dfSampleStats.set_index(miSampleCols.to_list()), on=miSampleCols.to_list()))
    
    logger.info('Ajout stats échantillons => {} opt-analyses pour rapport'.format(len(results)))

In [ ]:
# Compatibilité ascendante pour résultats d'avant fin août 2021 : ajout des infos 'runtime' (supposées).
if not optAnalysed and 'runtime' not in results.specs:
    
    oldRuntime = {'WARNING': 'Actually, not precisely what\'s listed below, but for sure something very close ...',
                  'platform': 'win32',
                  'cpython': '3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 07:34:03) [MSC v.1916 64 bit (AMD64)]',
                  'numpy': '1.19.4',
                  'pandas': '1.2.5',
                  'pickle': '4.0',
                  'zoopt': '0.4.0',
                  'matplotlib': '3.4.2',
                  'jinja2': '3.0.1'}
    
    results.updateSpecs(runtime=pd.Series(oldRuntime, name='Version'))
    
    logger.info('Ajout infos plateforme de calcul supposée')

## 3. Filtrage / sélection des résultats avant rapport

Au cas où, à ce stade, on désire supprimer les résultats pour certains échantillons, certaines analyses ...

In [ ]:
# Pour l'instant, pas de cas d'usage identifié

## 4. Rapports Excel et HTML auto-filtrants

In [ ]:
R = ads.MCDSTruncOptanalysisResultsSet

In [ ]:
# Super-synthesis: Select analysis results columns for the 3 textual columns on the left
sampleFilSorRepCols = \
[('header (head)', sampleNumCol, 'Value')] \
+ [('header (sample)', col, 'Value') for col in samplingCols] \
+ [('sample stats', 'total number of observations', 'Value'),
   ('sample stats', 'maximal observation distance', 'Value')]

paramFilSorRepCols = \
[
    R.CLParEstKeyFn, R.CLParEstAdjSer,
    #R.CLParEstSelCrit, R.CLParEstCVInt,
    R.CLParTruncLeft, R.CLParTruncRight, R.CLParModFitDistCuts,
]
    
resultFilSorRepCols = \
[
    ('header (head)', anlysIndCol, 'Value'),
    R.CLRunStatus,
    R.CLEffort, R.CLNObs, R.CLSightRate, R.CLNAdjPars,
    R.CLAic, R.CLChi2, R.CLKS, R.CLDCv,
    R.CLCmbQuaBal3, R.CLCmbQuaBal2, R.CLCmbQuaBal1,
     
    R.CLEswEdr,
    R.CLPDetec, R.CLDensity, R.CLDensityMin, R.CLDensityMax,
    R.CLNumber, R.CLNumberMin, R.CLNumberMax
]

In [ ]:
# Synthesis: Select columns of the filtered table
synthFilSorRepCols = \
[('header (head)', sampleNumCol, 'Value')] \
+ [('header (sample)', col, 'Value') for col in samplingCols] \
+ [('header (head)', anlysIndCol, 'Value'),
   ('header (tail)', 'FonctionClé', 'Value'),
   ('header (tail)', 'SérieAjust', 'Value'),
   ('header (tail)', 'TrGche', 'Value'),
   ('header (tail)', 'TrDrte', 'Value'),
   ('header (tail)', 'NbTrchMod', 'Value'),
 
   R.CLRunStatus,
   R.CLEffort, R.CLNTotObs, R.CLNObs, R.CLNAdjPars,
   R.CLDeltaAic, R.CLChi2, R.CLKS, R.CLCvMUw, R.CLCvMCw, R.CLDCv, 
   
   R.CLDensity, R.CLDensityMin, R.CLDensityMax,
   R.CLEswEdr, R.CLEswEdrMin, R.CLEswEdrMax,
   R.CLNumber, R.CLNumberMin, R.CLNumberMax,
   R.CLPDetec, R.CLPDetecMin, R.CLPDetecMax,

   R.CLSightRate,
   R.CLCmbQuaBal3, R.CLCmbQuaBal2, R.CLCmbQuaBal1,
   R.CLCmbQuaChi2, R.CLCmbQuaKS, R.CLCmbQuaDCv,

   R.CLGroupTruncLeft, R.CLGroupTruncRight,
   
   R.CLGrpOrdSmTrAic,
   R.CLGrpOrdClTrChi2KSDCv, #R.CLGrpOrdClTrChi2,
   R.CLGrpOrdClTrDCv,
   R.CLGrpOrdClTrQuaBal1, R.CLGrpOrdClTrQuaBal2, R.CLGrpOrdClTrQuaBal3, R.CLGrpOrdClTrQuaChi2,
   R.CLGrpOrdClTrQuaKS, R.CLGrpOrdClTrQuaDCv,
   R.CLGblOrdChi2KSDCv, R.CLGblOrdQuaBal1, R.CLGblOrdQuaBal2, R.CLGblOrdQuaBal3,
   R.CLGblOrdQuaChi2, R.CLGblOrdQuaKS, R.CLGblOrdQuaDCv,
   R.CLGblOrdDAicChi2KSDCv
]

In [ ]:
# Define filter and sort schemes to apply
whichBestQua = [R.CLGrpOrdClTrChi2KSDCv, R.CLGrpOrdClTrDCv, R.CLCmbQuaBal1, R.CLCmbQuaBal2, R.CLCmbQuaBal3,
                R.CLGrpOrdClTrQuaChi2, R.CLGrpOrdClTrQuaKS, R.CLGrpOrdClTrQuaDCv]

dupSubset = [R.CLNObs, R.CLEffort, R.CLDeltaAic, R.CLChi2, R.CLKS, R.CLCvMUw, R.CLCvMCw, R.CLDCv, 
             R.CLPDetec, R.CLPDetecMin, R.CLPDetecMax, R.CLDensity, R.CLDensityMin, R.CLDensityMax]
dDupRounds = {R.CLDeltaAic: 1, R.CLChi2: 2, R.CLKS: 2, R.CLCvMUw: 2, R.CLCvMCw: 2, R.CLDCv: 2, 
              R.CLPDetec: 3, R.CLPDetecMin: 3, R.CLPDetecMax: 3, R.CLDensity: 2, R.CLDensityMin: 2, R.CLDensityMax: 2}

whichFinalQua = R.CLCmbQuaBal3
ascFinalQua = False

filSorRepSchemes = [dict(method=R.filterSortOnExCAicMulQua,
                         deduplicate=dict(dupSubset=dupSubset, dDupRounds=dDupRounds),
                         filterSort=dict(sightRate=97.5, nBestAIC=2, nBestQua=1, whichBestQua=whichBestQua,
                                         nFinalRes=8, whichFinalQua=whichFinalQua, ascFinalQua=ascFinalQua),
                         preselCols=[R.CLCmbQuaBal1, R.CLCmbQuaBal2, R.CLCmbQuaBal3],
                         preselAscs=False, preselThrhs=0.4, preselNum=3),
                    dict(method=R.filterSortOnExCAicMulQua,
                         deduplicate=dict(dupSubset=dupSubset, dDupRounds=dDupRounds),
                         filterSort=dict(sightRate=95, nBestAIC=2, nBestQua=1, whichBestQua=whichBestQua,
                                         nFinalRes=10, whichFinalQua=whichFinalQua, ascFinalQua=ascFinalQua),
                         preselCols=[R.CLCmbQuaBal1, R.CLCmbQuaBal2, R.CLCmbQuaBal3],
                         preselAscs=False, preselThrhs=0.3, preselNum=4),
                    dict(method=R.filterSortOnExCAicMulQua,
                         deduplicate=dict(dupSubset=dupSubset, dDupRounds=dDupRounds),
                         filterSort=dict(sightRate=92.5, nBestAIC=3, nBestQua=1, whichBestQua=whichBestQua,
                                         nFinalRes=12, whichFinalQua=whichFinalQua, ascFinalQua=ascFinalQua),
                         preselCols=[R.CLCmbQuaBal1, R.CLCmbQuaBal2, R.CLCmbQuaBal3],
                         preselAscs=False, preselThrhs=0.2, preselNum=5),
                    dict(method=R.filterSortOnExecCode,
                         deduplicate=dict(dupSubset=dupSubset, dDupRounds=dDupRounds),
                         filterSort=dict(whichFinalQua=whichFinalQua, ascFinalQua=ascFinalQua),
                         preselCols=[R.CLCmbQuaBal1, R.CLCmbQuaBal2, R.CLCmbQuaBal3],
                         preselAscs=False, preselThrhs=0.1, preselNum=10)]

In [ ]:
# Define final sorting for report tables
sortFilSorRepCols = [('header (head)', sampleNumCol, 'Value'), whichFinalQua]
sortFilSorRepAscend = [True, False]

assert len(sortFilSorRepCols) == len(sortFilSorRepAscend)

In [ ]:
filSorReport = ads.MCDSResultsFilterSortReport(resultsSet=results,
                                               sampleCols=sampleFilSorRepCols, paramCols=paramFilSorRepCols,
                                               resultCols=resultFilSorRepCols, synthCols=synthFilSorRepCols,
                                               sortCols=sortFilSorRepCols, sortAscend=sortFilSorRepAscend,
                                               filSorSchemes=filSorRepSchemes, lang='fr', 
                                               title=titreEtude, description=descrEtude, anlysSubTitle='Détails',
                                               subTitle="Rapport d'analyse auto-filtré, méthode {fsId}",
                                               keywords=motsClesEtude, pySources=['Visionature-ds-points.ipynb'],
                                               tgtFolder=workDir,
                                               tgtPrefix=f'{nomEtude}{sousEtude}-OptAnalyses{varEtude}-rapport')

In [ ]:
%%time

xlsxFilSorRep = filSorReport.toExcel()

backup(xlsxFilSorRep)

logger.info('Rapport Excel filtré : ' + pl.Path(xlsxFilSorRep).as_posix())

In [ ]:
%%time

htmlFilSorScheme = next(schm for schm in filSorRepSchemes
                        if schm['method'] is R.filterSortOnExCAicMulQua and schm['filterSort']['sightRate'] == 92.5)

htmlFilSorRep = filSorReport.toHtml(htmlFilSorScheme)

backup(htmlFilSorRep)

afsId = results.filSorSchemeId(htmlFilSorScheme)
logger.info(f'Rapport HTML auto-filtré (méthode {afsId}):\n=> ' + pl.Path(htmlFilSorRep).resolve().as_uri())

In [ ]:
#%%time

# Pandas 1.1 permet l'export ODF, mais hélas, pas encore de support pour le styling (via Style) :-(
#odsAFXRep = report.toOpenDoc()
#
#logger.info('Rapport OpenDoc/Spreadsheet : ' + pl.Path(htmlFilSorRep).resolve().as_uri())

In [ ]:
os.startfile(xlsxFilSorRep)

In [ ]:
#os.startfile(odsAFXRep)

## 5. Rapports Excel et HTML simple

(rapport 'Full', toutes analyses, sans filtre)

In [ ]:
# Super-synthesis: Select analysis results columns for the 3 textual columns on the left
sampleRepCols = \
[('header (head)', sampleNumCol, 'Value')] \
+ [('header (sample)', col, 'Value') for col in samplingCols] \
+ [('sample stats', 'total number of observations', 'Value'),
   ('sample stats', 'maximal observation distance', 'Value')]

paramRepCols = \
[
    R.CLParEstKeyFn, R.CLParEstAdjSer,
    #R.CLParEstSelCrit, R.CLParEstCVInt,
    R.CLParTruncLeft, R.CLParTruncRight, R.CLParModFitDistCuts,
]
    
resultRepCols = \
[
    ('header (head)', anlysIndCol, 'Value'),
    R.CLRunStatus,
    R.CLEffort, R.CLNObs, R.CLSightRate, R.CLNAdjPars,
    R.CLAic, R.CLChi2, R.CLKS, R.CLDCv,
    R.CLCmbQuaBal3, R.CLCmbQuaBal2, R.CLCmbQuaBal1,
     
    R.CLEswEdr,
    R.CLPDetec, R.CLDensity, R.CLDensityMin, R.CLDensityMax,
    R.CLNumber, R.CLNumberMin, R.CLNumberMax
]

In [ ]:
# Sélection des colonnes pour les tableaux de synthèse du rapport
synthRepCols = \
[('header (head)', col, 'Value') for col in [anlysIndCol, sampleNumCol]] \
+ [('header (sample)', col, 'Value') for col in samplingCols] \
+ [R.CLParEstKeyFn, R.CLParEstAdjSer,
   #R.CLParEstSelCrit, R.CLParEstCVInt,
   R.CLParTruncLeft, R.CLParTruncRight, R.CLParModFitDistCuts,
   
   R.CLRunStatus,
   
   R.CLEffort, R.CLNObs, R.CLSightRate, R.CLNAdjPars,
   R.CLAic, R.CLChi2, R.CLKS, R.CLDCv,
   R.CLCmbQuaBal3, R.CLCmbQuaBal2, R.CLCmbQuaBal1,
    
   R.CLEswEdr,
   R.CLPDetec, R.CLDensity, R.CLDensityMin, R.CLDensityMax,
   R.CLNumber, R.CLNumberMin, R.CLNumberMax
   
   ('encounter rate', 'number of observations (n)', 'Value'),
   ('encounter rate', 'right truncation distance (w)', 'Value'),
   R.CLEffort,
   
   R.CLDeltaAic, R.CLAic, R.CLChi2, R.CLKS, R.CLDCv, R.CLCvMUw, R.CLCvMCw,
   
   R.CLEswEdr, CLEswEdrMin, CLEswEdrMax,   
   R.CLDensity, R.CLDensityMin, R.CLDensityMax, R.CLDeltaDCv,
   R.CLPDetec, R.CLPDetecMin, R.CLPDetecMax, R.CLPDetecDf,
   R.CLNumber, R.CLNumberMin, R.CLNumberMax, R.CLNumberDf,
   
   ('run output', 'run folder', 'Value')
]

In [ ]:
sortRepCols = \
[('header (head)', sampleNumCol, 'Value')] \
+ [('header (sample)', col, 'Value') for col in samplingCols] \
+ [R.CLParTruncLeft, R.CLParTruncRight,
   R.CLDeltaAic, R.CLChi2, R.CLKS,  # R.CLDCv,
   R.CLRunStatus]

sortRepAscend = [True]*(1+len(samplingCols)+3) + [False]*2 + [True]

assert len(sortRepCols) == len(sortRepAscend)

In [ ]:
report = ads.MCDSResultsFullReport(resultsSet=results, 
                                   sampleCols=sampleRepCols, paramCols=paramRepCols,
                                   resultCols=resultRepCols, synthCols=synthRepCols,
                                   sortCols=sortRepCols, sortAscend=sortRepAscend,
                                   title=titreEtude, subTitle='Rapport d\'analyse brut',
                                   anlysSubTitle='Détail de toutes les analyses', description=descrEtude,
                                   keywords=motsClesEtude, pySources=['Visionature-ds-points.ipynb'],
                                   lang='fr', plotImgSize=(768, 384),
                                   #plotImgQuality=80, plotImgFormat='jpg', # Same final size as raw PNG :-(
                                   tgtFolder=workDir, tgtPrefix=f'{nomEtude}{sousEtude}-OptAnalyses{varEtude}-rapport-brut')

In [ ]:
%%time

xlsxRep = report.toExcel()

backup(xlsxRep)

logger.info('Rapport Excel : ' + pl.Path(xlsxRep).as_posix())

In [ ]:
%%time

htmlRep = report.toHtml()

backup(htmlRep)

logger.info('Rapport HTML : ' + pl.Path(htmlRep).as_posix())

## 6. Comparaison aux résultats DS "manuels"

In [ ]:
cSclYellow, cSclOrange, cSclRed = '#fbfbba', '#f9da56', '#fe835a'
scaled2Colors = ['white', cSclRed]
scaled3Colors = ['white', cSclOrange, cSclRed]
scaled4Colors = ['white', cSclYellow, cSclOrange, cSclRed]

def scaledColorV(v, thresholds, colors, gt=True): # len(thresholds) == len(colors) - 1
        if pd.isnull(v):
            return cSclRed
        if gt:
            for ind, thresh in enumerate(thresholds):
                if abs(v) > thresh:
                    return colors[ind]
        else:
            for ind, thresh in enumerate(thresholds):
                if abs(v) < thresh:
                    return colors[ind]
        return colors[-1]

### a. ZPS Cantal 2020 : analyses manuelles de Mathis

In [ ]:
compCols = ['Nb données', 'GOF Chi-p', 'D CV', 'N', 'N LCL', 'N UCL']

In [ ]:
dfManuRep = pd.read_excel(dossier / 'CretesPlombCantalZPS2020-AnalysesD73Mathis-resultats.xlsx')
dfManuRep

In [ ]:
# Resultats auto : trié par meilleur AIC, colonnes traduites
results.sortRows(by=[col for col in sortRepCols if col in results.columns], ascending=sortRepAscend)

dfAutoRes = results.dfTransData('fr')
dfAutoRes

In [ ]:
# On ne garde que la (1ère) analyse de meilleur AIC
dfAutoResBest = dfAutoRes.groupby([colEspece, 'Adulte', 'Dist Tronc Drte', 'Tranch Dist Mod'],
                                  dropna=False, sort=False).first()

In [ ]:
# Chaîne courte d'identification d'un échantillon + paramètres troncature.
def echantTruncId(sRes):
    
    # Sample abbreviation
    abbrevs = [sampleAbbrev(sRes)]

    # Truncation parameters abbreviation
    dTroncAbbrv = { 'l': 'TrGche' if 'TrGche' in sRes.index else 'TroncGche',
                    'r': 'TrDrte' if 'TrDrte' in sRes.index else 'TroncDrte',
                    'm': 'NbTrches' if 'NbTrches' in sRes.index else 'NbTrModel'
                                    if 'NbTrModel' in sRes.index else  'NbTrchMod',
                    'd': 'NbTrDiscr' }
    for abbrev, name in dTroncAbbrv.items():
        if name in sRes.index and not pd.isnull(sRes[name]):
            abbrevs.append('{}{}'.format(abbrev, sRes[name][0].lower() if isinstance(sRes[name], str)
                                                 else int(sRes[name])))
   
    return '-'.join(abbrevs)

In [ ]:
# Mise en forme pour se rapprocher du tableau de résultats manuels
dfAutoResBest = dfAutoResBest.reindex(columns=['NObs', 'Chi2 P', 'CoefVar Densité',
                                               'Nombre', 'Min Nombre', 'Max Nombre'])
dfAutoResBest.reset_index(inplace=True)
dfAutoResBest.rename(columns={'NObs': 'Nb données', 'Chi2 P': 'GOF Chi-p', 'CoefVar Densité': 'D CV',
                              'Nombre': 'N', 'Min Nombre': 'N LCL', 'Max Nombre': 'N UCL',
                              'Dist Tronc Drte': 'TrDrte', 'Tranch Dist Mod': 'NbTrches' }, inplace=True)
dfAutoResBest.insert(0, 'Id EchTronc', dfAutoResBest.apply(echantTruncId, axis='columns'))
dfAutoResBest

In [ ]:
# Comparaison 1: Indicateur de proximité
# a. Calcul
dfProx = ads.ResultsSet.compareDataFrames(dfAutoResBest, dfManuRep, indexCols=['Id EchTronc'], subsetCols=compCols,
                                          dropCloser=np.inf)

# b. Ordre des lignes d'origine
dfProx = dfProx.join(dfAutoResBest.reset_index(drop=False)[['Id EchTronc', 'index']].set_index('Id EchTronc'))
dfProx.sort_values(by='index', inplace=True)
dfProx.drop(columns='index', inplace=True)

In [ ]:
# c. Colorisation
def proxColor(v):
    return 'background-color: ' + scaledColorV(v, thresholds=[1.4999, 0.9999, 0.5999], colors=scaled4Colors)
dfsProx = dfProx.reset_index().style.applymap(proxColor, subset=compCols).format('{:g}', subset=compCols)
dfsProx

In [ ]:
# Comparaison 2 : Différence colonne à colonne
# a. calcul
dfDiff = dfAutoResBest.join(dfManuRep[compCols + ['Id EchTronc']].set_index('Id EchTronc'),
                            on='Id EchTronc', rsuffix=' (m)')
dfDiff.rename(columns={nm: nm + ' (a)' for nm in compCols}, inplace=True)

for col in compCols:
    dfDiff[col + ' (a - m)'] = dfDiff[col + ' (a)'] - dfDiff[col + ' (m)']

# b. Ordre des colonnes
colOrder = headCols = ['Id EchTronc', colEspece, 'Adulte', 'TrDrte', 'NbTrches']
for col in compCols:
    colOrder += [col + ' (a)', col + ' (m)', col + ' (a - m)']
dfDiff = dfDiff.reindex(columns=colOrder)

In [ ]:
# c. Colorisation
def diffChi2Color(v):
    return 'background-color: ' + scaledColorV(v, thresholds=[0.01, 0.05], colors=scaled3Colors, gt=False)
def diffCvColor(v):
    return 'background-color: ' + scaledColorV(v, thresholds=[0.01, 0.05], colors=scaled3Colors, gt=False)
def diffNColor(v):
    return 'background-color: ' + scaledColorV(v, thresholds=[1e-12], colors=scaled2Colors, gt=False)
dfsDiff = dfDiff.style.applymap(diffChi2Color, subset=['GOF Chi-p (a - m)']) \
                      .applymap(diffCvColor, subset=['D CV (a - m)']) \
                      .applymap(diffNColor, subset=[col + ' (a - m)' for col in compCols if col.startswith('N')])
dfsDiff.format('{:g}', subset=[col for col in dfDiff.columns if col not in headCols])
dfsDiff

In [ ]:
with pd.ExcelWriter(dossier / 'CretesPlombCantalZPS2020-AnalysesMathis-ComparD73Auto.xlsx') as xlsWrtr:
    dfAutoResBest.to_excel(xlsWrtr, index=False, sheet_name='Auto Meilleur AIC')
    dfsProx.to_excel(xlsWrtr, index=False, sheet_name='ProximiteLog')
    dfsDiff.to_excel(xlsWrtr, index=False, sheet_name='Difference')
    dfAutoRes.to_excel(xlsWrtr, index=False, sheet_name='Auto Toutes')

# Annexe. Bilan & stats données individualisées

In [ ]:
print(f'{nomEtude}{sousEtude}')

## 1. Transects

In [ ]:
dfTransects

In [ ]:
dfTransects.Observateur.nunique()

In [ ]:
dfTransects[['Observateur', 'Passage', 'Point']].groupby(['Observateur', 'Passage']).count()

In [ ]:
dfTransects.Passage.value_counts()

## 2. Données individualisées

In [ ]:
dfObsCatIndiv['Durée'].replace('5mn', '05mn', inplace=True)

In [ ]:
dfObsCatIndiv

In [ ]:
# Nb total d'espèces 
display(dfObsCatIndiv['Espèce'].nunique())

# Nb d'espèces par passage et durée
df = dfObsCatIndiv[['Passage', 'Durée', 'Espèce']].groupby(['Passage', 'Durée']).nunique().unstack(-2).unstack(-1).to_frame().T
df.columns = df.columns.droplevel(0)
for p in ['a', 'b']:
    df[(p, '10mn/05mn')] = (df[(p, '10mn')] - df[(p, '05mn')]) / df[(p, '05mn')]
df.sort_index(axis='columns', inplace=True)
df

In [ ]:
# Liste des espèces contactées par passage et durée d'inventaire (tous adultes)
df = dfObsCatIndiv[['Passage', 'Durée', 'Espèce', 'Distance']].copy()
df['Adulte'] = adulte = 'Tous adultes'
df['Méthode'] = sousEtude[1:]

df = df.groupby(['Espèce', 'Méthode', 'Adulte', 'Passage', 'Durée']).count().unstack(-4).unstack(-3).unstack(-2).unstack(-1)
df.columns = df.columns.droplevel(0)
df.fillna(0, inplace=True)
for duree in ['05mn', '10mn']:
    df[(sousEtude[1:], adulte, 'b+a', duree)] = \
        df[(sousEtude[1:], adulte, 'a', duree)] + df[(sousEtude[1:], adulte, 'b', duree)]

df.sort_values(by=[(sousEtude[1:], adulte, 'b+a', '10mn')], ascending=False, inplace=True)
df.sort_index(axis='columns', inplace=True)
df1 = df
df

In [ ]:
# Liste des espèces contactées par passage et durée d'inventaire (mâles uniquement)
df = dfObsCatIndiv.loc[dfObsCatIndiv.Adulte == 'm', ['Passage', 'Durée', 'Espèce', 'Distance']].copy()
df['Adulte'] = adulte = 'Mâles uniquement'
df['Méthode'] = sousEtude[1:]

df = df.groupby(['Espèce', 'Méthode', 'Adulte', 'Passage', 'Durée']).count().unstack(-4).unstack(-3).unstack(-2).unstack(-1)
df.columns = df.columns.droplevel(0)

df.fillna(0, inplace=True)
for duree in ['05mn', '10mn']:
    df[(sousEtude[1:], adulte, 'b+a', duree)] = \
        df[(sousEtude[1:], adulte, 'a', duree)] + df[(sousEtude[1:], adulte, 'b', duree)]

df.sort_values(by=[(sousEtude[1:], adulte, 'b+a', '10mn')], ascending=False, inplace=True)
df.sort_index(axis='columns', inplace=True)
df

In [ ]:
# Liste des espèces contactées par passage et durée d'inventaire (tous adultes et mâles uniquement en vis à vis)
df1.join(df, how='outer').sort_values(by=[(sousEtude[1:], 'Tous adultes', 'b+a', '10mn')],
                                      ascending=False) # .to_excel(f'donnees/acdc/bilan-especes{sousEtude}.xlsx')

In [ ]:
# Idem, mais en ajoutant l'autre sous-étude en vis à vis
autreSousEtude = '-Pap' if 'Nat' in sousEtude else '-Nat'
df2 = pd.read_excel(f'donnees/acdc/bilan-especes{autreSousEtude}.xlsx', header=[0, 1, 2, 3], skiprows=[4], index_col=0)
df2 = df1.join(df, how='outer').join(df2, how='outer')
df3 = df2.sort_values(by=[(autreSousEtude[1:], 'Tous adultes', 'b+a', '10mn'), (sousEtude[1:], 'Tous adultes', 'b+a', '10mn')],
                      ascending=False) # .to_excel('donnees/acdc/bilan-especes.xlsx')
df3

In [ ]:
# Croisement des effectifs et des échantillons à opt-analyser (enfin ... pour les mâles a+b uniquement)
df4 = df3[[(autreSousEtude[1:], 'Mâles uniquement', 'b+a', '05mn'), (sousEtude[1:], 'Mâles uniquement', 'b+a', '05mn'),
           (autreSousEtude[1:], 'Mâles uniquement', 'b+a', '10mn'), (sousEtude[1:], 'Mâles uniquement', 'b+a', '10mn')]]
df4 = df4.stack(-2)
df4.sort_index(axis='columns', inplace=True)
df4

In [ ]:
df4.columns = [sousEtude[1:] + ' 05mn', sousEtude[1:] + ' 10mn',
               autreSousEtude[1:] + ' 05mn', autreSousEtude[1:] + ' 10mn']
df4.reset_index(inplace=True)
df4.rename(columns=dict(level_0='Espèce'), inplace=True)
df4['Adulte'] = 'm'
df4.Passage.replace('b+a', 'a+b', inplace=True)
df4

In [ ]:
dfExplOptAnlysSpecs = pd.read_excel(dossier / f'{nomEtude}{sousEtude}-ExplOptAnlysSpecs.xlsx', index_col=0)
dfSubEchants = dfExplOptAnlysSpecs[['Espèce', 'Passage', 'Adulte']].drop_duplicates()
dfSubEchants.join(df4.set_index(['Espèce', 'Passage', 'Adulte']),
                  on=['Espèce', 'Passage', 'Adulte']).to_excel('donnees/acdc/bilan-echants.xlsx')

In [ ]:
# Nbre d'individus adultes contactés par passage et durée d'inventaire
df = dfObsCatIndiv

df = df[['Passage', 'Durée', 'Adulte']].groupby(['Passage', 'Durée']).count().unstack(-2)
df.columns = df.columns.droplevel(0)
df.sort_index(axis='columns', inplace=True)

print('05mn => 10mn : +', 100 * (df.loc['10mn'].sum() - df.loc['05mn'].sum()) / df.loc['05mn'].sum(), '%')

df

In [ ]:
df = df.unstack(-1).to_frame().T
for p in ['a', 'b']:
    df[(p, '10mn/05mn')] = (df[(p, '10mn')] - df[(p, '05mn')]) / df[(p, '05mn')]
df.sort_index(axis='columns', inplace=True)
df

In [ ]:
# Nbre d'individus mâles contactés par passage et durée d'inventaire
df = dfObsCatIndiv[dfObsCatIndiv.Adulte == 'm']

df = df[['Passage', 'Durée', 'Adulte']].groupby(['Passage', 'Durée']).count().unstack(-2)
df.columns = df.columns.droplevel(0)

print('05mn => 10mn : +', 100 * (df.loc['10mn'].sum() - df.loc['05mn'].sum()) / df.loc['05mn'].sum(), '%')

df

In [ ]:
df = df.unstack(-1).to_frame().T
for p in ['a', 'b']:
    df[(p, '10mn/5mn')] = (df[(p, '10mn')] - df[(p, '05mn')]) / df[(p, '05mn')]
df.sort_index(axis='columns', inplace=True)
df

# Annexe. Tests non régression suite évolutions pyaudisam

## Non régression rapports auto-filtrés

* suite optimisation calcul indicateurs qualité 28/11/2021

### 1. Load reference and target = "current" report

In [ ]:
# Load reference report
fpnRefRep, cmpSfx = workDir / 'ACDC2019-Nat-OptAnalyses-rapport.q3.211104.ods', 'q3211104'
#fpnRefRep = workDir / 'ACDC2019-Nat-OptAnalyses-rapport.211121.xlsx'

ddfRefRep = pd.read_excel(fpnRefRep, sheet_name=None)

# Get "all results" sheet (ref report)
dfRefRes = ddfRefRep['Détails']

print(', '.join(ddfRefRep.keys()))

#snRefPrfx = 'AFSM-'
snRefPrfx = 'MFTA-'
{sn[len(snRefPrfx):]: len(ddfRefRep[sn]) for sn in ddfRefRep if sn.startswith(snRefPrfx)}

In [ ]:
# Load last generated report
#fpnCurRep, curSfx = workDir / 'ACDC2019-Nat-OptAnalyses-rapport.211121.xlsx', '1121'
fpnCurRep, curSfx = workDir / 'ACDC2019-Nat-OptAnalyses-rapport.211128.xlsx', '1128'

cmpSfx += curSfx

ddfCurRep = pd.read_excel(fpnCurRep, sheet_name=None)

# Get "all results" sheet (last report)
dfCurRes = ddfCurRep['Détails']

print(', '.join(ddfCurRep.keys()))

snCurPrfx = 'MFTA-'
{sn[len(snCurPrfx):]: len(ddfCurRep[sn]) for sn in ddfCurRep if sn.startswith(snCurPrfx)}

### 2. Compare quality indicators

(for all results, not only filterd ones)

In [ ]:
quaCols = [col for col in dfRefRes.columns if col.startswith('Qual ')]

dfRelDiff = ads.DataSet.compareDataFrames(dfRefRes, dfRes, indexCols=[anlysIndCol], subsetCols=quaCols,
                                          dropCloser=14, dropCloserCols=True)

dfRelDiff

In [ ]:
# Other comparison of QualBal1
dfComp = dfRefRes[['Qual Equi 1']].compare(dfRes[['Qual Equi 1']])
dfComp

In [ ]:
# Other comparison of QualBal1
dfComp = dfRefRes[['Qual Equi 3']].compare(dfRes[['Qual Equi 3']])
dfComp

In [ ]:
dfBadRes = dfRes.loc[dfComp.index]

#dfBadRes.to_excel('tmp/_.xlsx')

assert dfBadRes[~dfBadRes['Chi2 P'].isnull() & (dfBadRes['Chi2 P'] > 0.1)].empty

### 3. Compare lists of auto-filtered analyses and check differences

In [ ]:
sampParamCols = ['Echant', 'Espèce', 'Passage', 'Adulte', 'Durée',
                 'FonctionClé', 'SérieAjust', 'TrGche', 'TrDrte', 'NbTrchMod']
mcdsQuaCols = ['Taux Obs', 'NbPars SérAjust', 'Delta AIC', 'Chi2 P', 'KS P', 'CvM Uw P', 'CvM Cw P', 'CoefVar Densité']
balQuaCols = ['Qual Equi 1', 'Qual Equi 2', 'Qual Equi 3']  # , 'Qual Chi2+', 'Qual KS+', 'Qual DCv+']

In [ ]:
# Get 1 reference auto-filtered sheet
snAfsRefRes = 'ExAicMQua-r925d12'  # q3.211104
#snAfsRefRes = 'ExAicMQua-r925m8q3d12'  # 211121 and on ...

dfAfsRefRes = ddfRefRep[snRefPrfx + snAfsRefRes]

dfAfsRefRes[[anlysIndCol] + sampParamCols + mcdsQuaCols + balQuaCols]

In [ ]:
# Get 1 last report auto-filtered sheet
snAfsCurRes = 'ExAicMQua-r925m8q3d12'  # 211121 and on ...
cmpSfx += '-' + snAfsCurRes.split('-')[-1]
dfAfsCurRes = ddfCurRep[snPrfx + snAfsCurRes]

dfAfsCurRes[[anlysIndCol] + sampParamCols + mcdsQuaCols + balQuaCols]

In [ ]:
dfKptAnls = dfRefRes[[anlysIndCol] + sampParamCols + mcdsQuaCols + balQuaCols].copy()

dfKptAnls.loc[dfKptAnls[anlysIndCol].isin(dfAfsRefRes[anlysIndCol]), 'Ref'] = 1
dfKptAnls.loc[dfKptAnls[anlysIndCol].isin(dfAfsCurRes[anlysIndCol]), 'Cur'] = 1

dfKptAnls = dfKptAnls.join(dfCurRes[[anlysIndCol] + balQuaCols].set_index(anlysIndCol), on=anlysIndCol, rsuffix=' Cur')
for quaCol in balQuaCols:
    dfKptAnls[quaCol + ' Cur - Ref'] = dfKptAnls[quaCol] - dfKptAnls[quaCol + ' Cur']

dfKptAnls = dfKptAnls[(dfKptAnls.Ref == 1) | (dfKptAnls.Cur == 1)]

dfKptAnls.sort_values(by=['Echant', 'TrGche', 'TrDrte'], ascending=True, na_position='first', inplace=True)

dfKptAnls.to_excel(workDir / f'{nomEtude}{sousEtude}{varEtude}-nonreg.{cmpSfx}.xlsx', index=False)

dfKptAnls

## Non régression ACDC 2019 suite correction décodage effectifs 09/2020

In [ ]:
fpn = dossier / f'{nomEtude}{sousEtude}-ObsIndivDist.xlsx'

with pd.ExcelFile(fpn) as xlsFile:
    
    dfNewObs = pd.read_excel(xlsFile, sheet_name='Donnees')
    dfNewInv = pd.read_excel(xlsFile, sheet_name='Inventaires')

dict(etude=nomEtude, donnees=len(dfNewObs), inventaires=len(dfNewInv))

In [ ]:
fpn = dossier / 'ACDC2019-Naturalist-ObsIndivAvecDist.deprec.xlsx'

with pd.ExcelFile(fpn) as xlsFile:
    
    dfOldObs = pd.read_excel(xlsFile, sheet_name='Donnees')
    dfOldInv = pd.read_excel(xlsFile, sheet_name='Inventaires')

dict(etude=nomEtude, donnees=len(dfOldObs), inventaires=len(dfOldInv))

### 1. Obs. individualisées

In [ ]:
#dfNewObs = dfObsCatIndiv.copy()

In [ ]:
dfNewObs = dfNewObs[['Observateur', 'Point', 'Passage', 'Date', 'Horaire', 'Espèce', 'Distance', 'Adulte', 'Durée']]
dfNewObs['DateHeure'] = dfNewObs[['Date', 'Horaire']].apply(lambda s: pd.Timestamp(s.Date) + pd.Timedelta(s.Horaire + ':00'),
                                                            axis='columns')
dfNewObs.Distance = dfNewObs.Distance.round(6)
dfNewObs = dfNewObs.reindex(columns=['Observateur', 'Point', 'Passage', 'DateHeure', 'Espèce', 'Distance', 'Adulte', 'Durée'])
dfNewObs.sort_values(by=list(dfNewObs.columns), inplace=True)
dfNewObs.reset_index(inplace=True, drop=True)
dfNewObs

In [ ]:
dfOldObs.Distance = dfOldObs.Distance.round(6)
dfOldObs.sort_values(by=list(dfNewObs.columns), inplace=True)
dfOldObs.reset_index(inplace=True, drop=True)
dfOldObs

In [ ]:
dfNewObs.equals(dfOldObs)

In [ ]:
dfNewObs[dfNewObs.Durée != dfOldObs.Durée]

In [ ]:
dfRawObs = vnds.sightings()

In [ ]:
dfRawObs.loc[(dfRawObs.Observateur == 'Romain Riols') & (dfRawObs['Espèce'] == 'Passer domesticus')
             & (dfRawObs.Distance.round(1).isin([42.6, 57.0, 85.6])), obsBrutesColsAff]

In [ ]:
dfRawObs.loc[(dfRawObs.Observateur == 'Romain Riols') & (dfRawObs['Espèce'] == 'Passer domesticus')
              & (dfRawObs['Code atlas'] == 5), obsBrutesColsAff]

### 2. Inventaires

In [ ]:
#dfNewInv = dfInventaires.copy()

In [ ]:
dfNewInv = dfNewInv.drop(columns=['ID liste']).sort_values(by=['Point', 'Passage']).reset_index(drop=True)
dfNewInv['Effort'] = effortConst
dfNewInv

In [ ]:
dfOldInv = dfOldInv[dfNewInv.columns].sort_values(by=['Point', 'Passage']).reset_index(drop=True)
dfOldInv

In [ ]:
assert dfNewInv.equals(dfOldInv)

## Non régression ZPS Cantal 2020 suite généralisation NB 11/2020

In [ ]:
repCols4Comp = ['Echant',
                'NObs', 'Effort', 'AIC', 'Chi2 P', 'KS P',
                'CoefVar Densité', 'EDR/ESW', 'Min EDR/ESW',
                'Max EDR/ESW', 'Densité', 'Min Densité', 'Max Densité',
                'PDetec', 'Min PDetec', 'Max PDetec',
                'DegLib PDetec']

In [ ]:
dfNewRep = pd.read_excel(dossier / '201116-1938/CretesPlombCantalZPS2020-OptAnalyses-CommeMathis-rapport.xlsx', index_col=0)
dfNewRep = dfNewRep.set_index('Analyse', drop=True)[repCols4Comp].sort_index()
dfNewRep

In [ ]:
dfOldRep = pd.read_excel(dossier / '201010-2206/CretesPlombCantalZPS2020-OptAnalyses-CommeMathis-rapport.xlsx', index_col=0)
dfOldRep = dfOldRep.set_index('Analyse', drop=True)[repCols4Comp].sort_index()
dfOldRep

In [ ]:
dfNewRep.equals(dfOldRep)

In [ ]:
dfNewRep.compare(dfOldRep)

# Bac à sable